In [75]:
from __future__ import division # For python 2.*

import numpy as np
import matplotlib.pyplot as plt
import mltools as ml
from sklearn import *
from sklearn.metrics import roc_curve, auc

np.random.seed(0)
%matplotlib inline

In [329]:
X = np.genfromtxt('./uci-cs178-f19/X_train.txt', delimiter=',')
Y = np.genfromtxt('./uci-cs178-f19/Y_train.txt', delimiter=',')
X,Y = ml.shuffleData(X,Y)

print X.shape
print Y.shape
Xtr, Xva, Ytr, Yva = ml.splitData(X, Y, 0.80)
Xte = np.genfromtxt('./uci-cs178-f19/X_test.txt',delimiter=',')

Xtr, param = ml.transforms.rescale(Xtr)
Xva, _ = ml.transforms.rescale(Xva,param)
Xte, _ = ml.transforms.rescale(Xte,param)

(7423, 107)
(7423,)


# kNN

In [330]:
## knn
from sklearn.neighbors import KNeighborsClassifier
kneighbors = [5,15,30,50,75,100,150,300]

for k in kneighbors:
    knn = KNeighborsClassifier(n_neighbors=k, n_jobs=-1).fit(Xtr, Ytr)
    knn_tr_pred = knn.predict_proba(Xtr)
    knn_va_pred = knn.predict_proba(Xva)
    
    fpr_tr, tpr_tr, thresholds = roc_curve(Ytr, knn_tr_pred[:,1])
    knn_tr_auc = metrics.auc(fpr_tr, tpr_tr)
    
    fpr_va, tpr_va, thresholds = roc_curve(Yva, knn_va_pred[:,1])
    knn_va_auc = metrics.auc(fpr_va, tpr_va)
    
    print "K=" + str(k) + ""
    print "Training AUC: " + str(knn_tr_auc)
    print "Validation AUC: " + str(knn_va_auc)

K=5
Training AUC: 0.7913753309232077
Validation AUC: 0.5844711941549886
K=15
Training AUC: 0.7157017274282228
Validation AUC: 0.6116546347412319
K=30
Training AUC: 0.6866081188309378
Validation AUC: 0.6098752527067296
K=50
Training AUC: 0.6762196154390882
Validation AUC: 0.6166153086742377
K=75
Training AUC: 0.6625832755314744
Validation AUC: 0.6146009139181974
K=100
Training AUC: 0.654497956810209
Validation AUC: 0.618098883190511
K=150
Training AUC: 0.6459438302114274
Validation AUC: 0.6193746665360033
K=300
Training AUC: 0.6359149050717953
Validation AUC: 0.6104514421977592


In [289]:
from sklearn.feature_selection import SelectKBest, f_classif
bestk = [10,20,30,40,50,60,70,80,90,100]

for bk in bestk:
    test = SelectKBest(score_func=f_classif, k=bk)
    fit = test.fit(X, Y)
    features = fit.transform(X)

    Xtr, Xva, Ytr, Yva = ml.splitData(features, Y, 0.80)

    knn = KNeighborsClassifier(n_neighbors=300).fit(Xtr,Ytr)
    knn_tr_pred = knn.predict_proba(Xtr)
    knn_va_pred = knn.predict_proba(Xva)

    fpr_tr, tpr_tr, thresholds = roc_curve(Ytr, knn_tr_pred[:,1])
    knn_tr_auc = metrics.auc(fpr_tr, tpr_tr)

    fpr_va, tpr_va, thresholds = roc_curve(Yva, knn_va_pred[:,1])
    knn_va_auc = metrics.auc(fpr_va, tpr_va)

    print "K="+str(bk)
    print "Training AUC: " + str(knn_tr_auc)
    print "Validation AUC: " + str(knn_va_auc)

K=10
Training AUC: 0.5780580030022483
Validation AUC: 0.556532522006691
K=20
Training AUC: 0.5780580030022483
Validation AUC: 0.5565805992786595
K=30
Training AUC: 0.5782419704152263
Validation AUC: 0.5564980515098078
K=40
Training AUC: 0.5941967045610417
Validation AUC: 0.5560308855652073
K=50
Training AUC: 0.5938884471074116
Validation AUC: 0.5561723960260959
K=60
Training AUC: 0.5966218069060334
Validation AUC: 0.552789751739853
K=70
Training AUC: 0.594014495701811
Validation AUC: 0.561393769185553
K=80
Training AUC: 0.5958276213427612
Validation AUC: 0.5614463820492165
K=90
Training AUC: 0.5951211932842307
Validation AUC: 0.5627952670193541
K=100
Training AUC: 0.5951165983534717
Validation AUC: 0.5627952670193541


In [308]:
knn = KNeighborsClassifier(n_neighbors=100, n_jobs=-1).fit(Xtr, Ytr)
knn_tr_pred = knn.predict_proba(Xtr)
knn_va_pred = knn.predict_proba(Xva)

fpr_tr, tpr_tr, thresholds = roc_curve(Ytr, knn_tr_pred[:,1])
knn_tr_auc = metrics.auc(fpr_tr, tpr_tr)

fpr_va, tpr_va, thresholds = roc_curve(Yva, knn_va_pred[:,1])
knn_va_auc = metrics.auc(fpr_va, tpr_va)

print "Training AUC: " + str(knn_tr_auc)
print "Validation AUC: " + str(knn_va_auc)

Training AUC: 0.6555342754688316
Validation AUC: 0.6273204833788633


In [388]:
knn = KNeighborsClassifier(n_neighbors=100, n_jobs=-1).fit(Xtr, Ytr)
Pte_knn = knn.predict_proba(Xte)
Pva_knn = knn.predict_proba(Xva)

# Xte = np.genfromtxt('./uci-cs178-f19/X_test.txt', delimiter=',')
# Yte = np.vstack((np.arange(len(pred_soft)) , pred_soft[:,1]) ).T
# np.savetxt('knn.txt',Yte,'%d, %.2f',comments='',header='Id,Predicted',delimiter=',')

# Random Forest

In [153]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_curve, auc
from sklearn import tree
from sklearn import gaussian_process
from sklearn import linear_model

In [298]:
rf = RandomForestClassifier(n_estimators=1000, max_features=106, bootstrap=True, max_depth=10)
rf.fit(Xtr, Ytr)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=10, max_features=106, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [299]:
pred_soft = rf.predict_proba(Xtr)
false_positive_rate, true_positive_rate, thresholds = roc_curve(Ytr, pred_soft[:,1])
roc_auc = auc(false_positive_rate, true_positive_rate)
print roc_auc

0.9476643563277867


In [300]:
pred_soft = rf.predict_proba(Xva)
false_positive_rate, true_positive_rate, thresholds = roc_curve(Yva, pred_soft[:,1])
roc_auc = auc(false_positive_rate, true_positive_rate)
print roc_auc

0.7429059165436094


In [189]:
max = 0
currk = 0

for mf in range(10,108):
    rf = RandomForestClassifier(n_estimators=50, max_features=mf, bootstrap=True, max_depth=20)
    rf.fit(Xtr, Ytr)
    
    pred_soft = rf.predict_proba(Xtr)
    false_positive_rate, true_positive_rate, thresholds = roc_curve(Ytr, pred_soft[:,1])
    roc_auc_tr = auc(false_positive_rate, true_positive_rate)
    
    pred_soft = rf.predict_proba(Xva)
    false_positive_rate, true_positive_rate, thresholds = roc_curve(Yva, pred_soft[:,1])
    roc_auc_va = auc(false_positive_rate, true_positive_rate)
    
    if roc_auc_va > max:
        max = roc_auc_va
        currk = mf
    
    print "maxF = " + str(mf)
    print "Training: " + str(roc_auc_tr)
    print "Validation: " + str(roc_auc_va)
    
print max, currk

maxF = 10
Training: 0.999999205744059
Validation: 0.7367514592621883
maxF = 11
Training: 1.0
Validation: 0.7405796417960262
maxF = 12
Training: 1.0
Validation: 0.7410340174059414
maxF = 13
Training: 0.9999999999999999
Validation: 0.7384737332806101
maxF = 14
Training: 1.0
Validation: 0.7364612393117433
maxF = 15
Training: 1.0
Validation: 0.7418820038236479
maxF = 16
Training: 0.9999998865348656
Validation: 0.7523743619695777
maxF = 17
Training: 1.0
Validation: 0.7399719937747821
maxF = 18
Training: 1.0
Validation: 0.7415754590009903
maxF = 19
Training: 0.9999997730697311
Validation: 0.7483820237762695
maxF = 20
Training: 1.0
Validation: 0.7472946058994461
maxF = 21
Training: 0.9999997730697312
Validation: 0.7498430998392908
maxF = 22
Training: 1.0
Validation: 0.7477444468226357
maxF = 23
Training: 1.0
Validation: 0.7514320540679766
maxF = 24
Training: 1.0
Validation: 0.7417223828509031
maxF = 25
Training: 1.0
Validation: 0.7566850351710304
maxF = 26
Training: 0.9999997730697312
Validat

In [190]:
max = 0
currk = 0

for mf in range(10,108):
    rf = RandomForestClassifier(n_estimators=100, max_features=mf, bootstrap=True, max_depth=20)
    rf.fit(Xtr, Ytr)
    
    pred_soft = rf.predict_proba(Xtr)
    false_positive_rate, true_positive_rate, thresholds = roc_curve(Ytr, pred_soft[:,1])
    roc_auc_tr = auc(false_positive_rate, true_positive_rate)
    
    pred_soft = rf.predict_proba(Xva)
    false_positive_rate, true_positive_rate, thresholds = roc_curve(Yva, pred_soft[:,1])
    roc_auc_va = auc(false_positive_rate, true_positive_rate)
    
    if roc_auc_va > max:
        max = roc_auc_va
        currk = mf
    
    print "maxF = " + str(mf)
    print "Training: " + str(roc_auc_tr)
    print "Validation: " + str(roc_auc_va)
    
print max, mf

maxF = 10
Training: 1.0
Validation: 0.7403728600813341
maxF = 11
Training: 1.0
Validation: 0.7401932864869962
maxF = 12
Training: 0.9999999999999999
Validation: 0.747675519584405
maxF = 13
Training: 0.9999999999999999
Validation: 0.7560937120219987
maxF = 14
Training: 1.0
Validation: 0.7512515735362939
maxF = 15
Training: 1.0
Validation: 0.7507536649338116
maxF = 16
Training: 0.9999999999999999
Validation: 0.7524006631525868
maxF = 17
Training: 1.0
Validation: 0.7520424229012563
maxF = 18
Training: 1.0
Validation: 0.7555178068078345
maxF = 19
Training: 1.0
Validation: 0.7574486764156385
maxF = 20
Training: 1.0
Validation: 0.7488400271355653
maxF = 21
Training: 1.0
Validation: 0.7594657050712309
maxF = 22
Training: 1.0
Validation: 0.7576799454386494
maxF = 23
Training: 1.0
Validation: 0.7572400808262563
maxF = 24
Training: 1.0
Validation: 0.7531017257203804
maxF = 25
Training: 1.0
Validation: 0.7503727512488527
maxF = 26
Training: 1.0
Validation: 0.7494830457132698
maxF = 27
Training: 1

In [231]:
max = 0
currk = 0
depths=[10,20,30,40,50,60,70]
for d in depths:
    rf = RandomForestClassifier(n_estimators=100, max_features=106, bootstrap=True, max_depth=d)
    rf.fit(Xtr, Ytr)
    
    pred_soft = rf.predict_proba(Xtr)
    false_positive_rate, true_positive_rate, thresholds = roc_curve(Ytr, pred_soft[:,1])
    roc_auc_tr = auc(false_positive_rate, true_positive_rate)
    
    pred_soft = rf.predict_proba(Xva)
    false_positive_rate, true_positive_rate, thresholds = roc_curve(Yva, pred_soft[:,1])
    roc_auc_va = auc(false_positive_rate, true_positive_rate)
    
    if roc_auc_va > max:
        max = roc_auc_va
        currk = d
    
    print "depth = " + str(d)
    print "Training: " + str(roc_auc_tr)
    print "Validation: " + str(roc_auc_va)
    
print max, currk

depth = 10
Training: 0.9507687262857868
Validation: 0.7616133327045234
depth = 20
Training: 1.0
Validation: 0.754668913452783
depth = 30
Training: 1.0
Validation: 0.7530355192941851
depth = 40
Training: 0.9999999999999999
Validation: 0.7537411165487043
depth = 50
Training: 1.0
Validation: 0.7470125483851073


KeyboardInterrupt: 

In [385]:
rf = RandomForestClassifier(n_estimators=1000, max_features=106, bootstrap=True, max_depth=10)
rf.fit(Xtr, Ytr)
Pte_rf = rf.predict_proba(Xte)
Pva_fr = rf.predict_proba(Xva)

# Neural Network

In [390]:
X = np.genfromtxt('./uci-cs178-f19/X_train.txt', delimiter=',')
Y = np.genfromtxt('./uci-cs178-f19/Y_train.txt', delimiter=',')
X,Y = ml.shuffleData(X,Y)


Xtr, Xva, Ytr, Yva = ml.splitData(X, Y, 0.80)
Xte = np.genfromtxt('./uci-cs178-f19/X_test.txt',delimiter=',')

Xtr, param = ml.transforms.rescale(Xtr)
Xva, _ = ml.transforms.rescale(Xva,param)
Xte, _ = ml.transforms.rescale(Xte,param)

In [380]:
from sklearn.neural_network import MLPClassifier
nnet = MLPClassifier(solver='lbfgs', hidden_layer_sizes=2, max_iter=256, random_state=1)
nnet.fit(Xtr, Ytr)

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=2, learning_rate='constant',
       learning_rate_init=0.001, max_iter=256, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=1, shuffle=True, solver='lbfgs', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False)

In [381]:
pred_soft = nnet.predict_proba(Xtr)
false_positive_rate, true_positive_rate, thresholds = roc_curve(Ytr, pred_soft[:,1])
roc_auc_tr = auc(false_positive_rate, true_positive_rate)

pred_soft = nnet.predict_proba(Xva)
false_positive_rate, true_positive_rate, thresholds = roc_curve(Yva, pred_soft[:,1])
roc_auc_va = auc(false_positive_rate, true_positive_rate)

print "Training: " + str(roc_auc_tr)
print "Validation: " + str(roc_auc_va)

Training: 0.7298712148520866
Validation: 0.7100584793200146


In [360]:
layers = [2,3,4,5,6,7,8,9,10]
max = 0
currL = 0
for l in layers:
    nnet = MLPClassifier(solver='lbfgs', hidden_layer_sizes=l, random_state=1)
    nnet.fit(Xtr, Ytr)
    
    pred_soft = nnet.predict_proba(Xtr)
    false_positive_rate, true_positive_rate, thresholds = roc_curve(Ytr, pred_soft[:,1])
    roc_auc_tr = auc(false_positive_rate, true_positive_rate)

    pred_soft = nnet.predict_proba(Xva)
    false_positive_rate, true_positive_rate, thresholds = roc_curve(Yva, pred_soft[:,1])
    roc_auc_va = auc(false_positive_rate, true_positive_rate)

    if roc_auc_va > max:
        max = roc_auc_va
        currL = l
    
    print "Layers=" + str(l)
    print "Training: " + str(roc_auc_tr)
    print "Validation: " + str(roc_auc_va)

print "Best # Layers: ", currL
print "AUC: ", max

Layers=2
Training: 0.7238576747281769
Validation: 0.7100847805030237
Layers=3
Training: 0.7107653935882122
Validation: 0.6442792206143231
Layers=4
Training: 0.7593065800890262
Validation: 0.7079707095515013
Layers=5
Training: 0.7688801128196154
Validation: 0.682365147486151
Layers=6
Training: 0.7645093625462521
Validation: 0.6663468200962805
Layers=7
Training: 0.7894164326186116
Validation: 0.6554925939496395
Layers=8
Training: 0.8045307573353685
Validation: 0.6554626650172499
Layers=9
Training: 0.814179857150978
Validation: 0.6461865098511534
Layers=10
Training: 0.8154567820380897
Validation: 0.662140444689519
Best # Layers:  2
AUC:  0.7100847805030237


In [375]:
maxIter = [2,4,8,16,32,64,128,256,512,1024,2048,4096,9192]
currMax = 0
currI = 0
for mi in maxIter:
    nnet = MLPClassifier(solver='lbfgs', hidden_layer_sizes=2, max_iter=mi, random_state=1)
    nnet.fit(Xtr, Ytr)
    
    pred_soft = nnet.predict_proba(Xtr)
    false_positive_rate, true_positive_rate, thresholds = roc_curve(Ytr, pred_soft[:,1])
    roc_auc_tr = auc(false_positive_rate, true_positive_rate)

    pred_soft = nnet.predict_proba(Xva)
    false_positive_rate, true_positive_rate, thresholds = roc_curve(Yva, pred_soft[:,1])
    roc_auc_va = auc(false_positive_rate, true_positive_rate)

    if roc_auc_va > currMax:
        currMax = roc_auc_va
        currI = mi
    
    print "MaxIter=" + str(mi)
    print "Training: " + str(roc_auc_tr)
    print "Validation: " + str(roc_auc_va)

print "Best # Layers: ", currI
print "AUC: ", currMax

MaxIter=2
Training: 0.5203886462837504
Validation: 0.4844414898441156
MaxIter=4
Training: 0.5517421219895565
Validation: 0.5175782596235121
MaxIter=8
Training: 0.625938277122655
Validation: 0.5911562725600665
MaxIter=16
Training: 0.6721692779860033
Validation: 0.6611573246073867
MaxIter=32
Training: 0.6908855202912941
Validation: 0.659751571722419
MaxIter=64
Training: 0.7001142923794906
Validation: 0.6740893442117445
MaxIter=128
Training: 0.7092966009221682
Validation: 0.6813729580305674
MaxIter=256
Training: 0.7298712148520866
Validation: 0.7100584793200146
MaxIter=512
Training: 0.7340312055752625
Validation: 0.7089656198191204
MaxIter=1024
Training: 0.7347623681816733
Validation: 0.7092123067769986
MaxIter=2048
Training: 0.7347623681816733
Validation: 0.7092123067769986
MaxIter=4096
Training: 0.7347623681816733
Validation: 0.7092123067769986
MaxIter=9192
Training: 0.7347623681816733
Validation: 0.7092123067769986
Best # Layers:  256
AUC:  0.7100584793200146


In [387]:
nnet = MLPClassifier(solver='lbfgs', hidden_layer_sizes=2, max_iter=256, random_state=1)
nnet.fit(Xtr, Ytr)
Pte_nnet = nnet.predict_proba(Xte)
Pva_nnet = nnet.predict_proba(Xva)

# Xte = np.genfromtxt('./uci-cs178-f19/X_test.txt', delimiter=',')
# Yte = np.vstack((np.arange(len(pred_soft)) , pred_soft[:,1]) ).T
# np.savetxt('neural_net.txt',Yte,'%d, %.2f',comments='',header='Id,Predicted',delimiter=',')

# Overall Ensemble 

In [439]:
from sklearn.ensemble import VotingClassifier

nnet = MLPClassifier(solver='lbfgs', hidden_layer_sizes=2, max_iter=256, random_state=1)
rf = RandomForestClassifier(n_estimators=1000, max_features=106, bootstrap=True, max_depth=10)
knn = KNeighborsClassifier(n_neighbors=100, n_jobs=-1).fit(Xtr, Ytr)

vote = VotingClassifier(estimators=[('rf', rf), ('knn', knn), ('nnet', nnet)], weights=[3,1,2],voting='soft')
vote.fit(Xtr, Ytr)

VotingClassifier(estimators=[('rf', RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=10, max_features=106, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_...True, solver='lbfgs', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False))],
         flatten_transform=None, n_jobs=None, voting='soft',
         weights=[3, 1, 2])

In [440]:
from sklearn.metrics import roc_auc_score

pred_soft = vote.predict_proba(Xtr)
false_positive_rate, true_positive_rate, thresholds = roc_curve(Ytr, pred_soft[:,1])
roc_auc_tr = auc(false_positive_rate, true_positive_rate)

pred_soft = vote.predict_proba(Xva)
false_positive_rate, true_positive_rate, thresholds = roc_curve(Yva, pred_soft[:,1])
roc_auc_va = auc(false_positive_rate, true_positive_rate)

pred_soft = vote.predict_proba(Xte)
# roc_auc_te = pred_soft.auc

print "Training: " + str(roc_auc_tr)
print "Validation: " + str(roc_auc_va)
# print "Test: " + str(roc_auc_score(y_true, y_scores))

Training: 0.9055175105399065
Validation: 0.7414809116540331


In [441]:
pred_soft = vote.predict_proba(Xte)
Xte = np.genfromtxt('./uci-cs178-f19/X_test.txt', delimiter=',')
Yte = np.vstack((np.arange(len(pred_soft)) , pred_soft[:,1]) ).T
np.savetxt('vote.txt',Yte,'%d, %.2f',comments='',header='Id,Predicted',delimiter=',')